In [1]:
 ####### Setup
import os
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage,AnnotationBbox,TextArea
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cv2
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import preprocessing
while os.getcwd().split('\\')[-1] != 'ecological-networks':
    %cd ..
import setup_paths
setup_paths.add_path()
from Species_Reintroductions import *
global DATA_TYPE
DATA_TYPE = 'Real_2D'
CASE_TO_SOLVE_init(DATA_TYPE)
get_CASE()

def CASE_init(CASE_TO_SOLVE):
    DATA_TYPE = CASE_TO_SOLVE
    CASE_TO_SOLVE_init(DATA_TYPE)
    NetworkName_Solved_df = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl"))
    M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))
    if DATA_TYPE.split('_')[0] == 'Syn':
        M = {k:M[k] for k in M.keys() if int(k.split('_')[2]) == 100}
    return DATA_TYPE, NetworkName_Solved_df, M

def init_HeatmapData(Case, Start = 0, End = -1):
    if End == -1:
        End = Start + 1
    if Case == 1:
        Heatmap01_All = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"Heatmap01_{DATA_TYPE}.pkl"))
        return {k:Heatmap01_All[k][Start:End] for k in Heatmap01_All.keys()}
    if Case == 2:
        Heatmap02_All = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"Heatmap02_{DATA_TYPE}.pkl"))
        return {k:Heatmap02_All[k][Start:End] for k in Heatmap02_All.keys()}
    if Case == 3:
        Heatmap03_All = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"Heatmap03_{DATA_TYPE}.pkl"))
        return {k:Heatmap03_All[k][Start:End] for k in Heatmap03_All.keys()}

d:\Github\ecological-networks\Code\01_Species_Reintroductions
d:\Github\ecological-networks\Code
d:\Github\ecological-networks
Real_2D


In [4]:
####### Scatter Plot

def get_ScatterPlot(SHOW_PLOT = True, set_dpi = 100):
    FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', f"Scatter_01_Consolidated.pkl")
    D01 = UnpickleObj(FileName)[4]
    D01.drop(D01[ D01['N'] == 0 ].index ,inplace = True)
    D01.dropna(axis = 0, inplace = True)
    D01 = D01[D01['Abundance'] > 1.0]
    D01.reset_index(drop = True, inplace = True)

    FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', f"Scatter_02_Consolidated.pkl")
    D02 = UnpickleObj(FileName)[4]
    D02.drop(D02[ D02['N'] == 0 ].index ,inplace = True)
    D02.dropna(axis = 0, inplace = True)
    D02 = D02[D02['Abundance'] > 1.0]
    D02.reset_index(drop = True, inplace = True)

    FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', f"Scatter_03_Consolidated.pkl")
    D03 = UnpickleObj(FileName)[4]
    D03.drop(D03[ D03['N'] == 0 ].index ,inplace = True)
    D03.dropna(axis = 0, inplace = True)
    D03 = D03[D03['Abundance'] > 0.3]
    D03.reset_index(drop = True, inplace = True)

    FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', f"Scatter_04_Consolidated.pkl")
    D04 = UnpickleObj(FileName)[4]
    D04.drop(D04[ D04['N'] == 0 ].index ,inplace = True)
    D04.dropna(axis = 0, inplace = True)
    D04 = D04[D04['Abundance'] > 0.3]
    D04.reset_index(drop = True, inplace = True)
    
    D_1D = pd.concat([D01,D02], ignore_index = True)
    D_2D = pd.concat([D04,D03], ignore_index = True)
    del D01,D02,D03,D04
    
    D_lst = list([D_1D, D_2D])
    V_lst = ['S','A','C','N']
    
    D1 = D_1D.copy()
    D2 = D_2D.copy()
    
    for Col_Name in list(D_1D.columns):
        
        X_scaled = preprocessing.MinMaxScaler().fit_transform(np.array(D1[Col_Name]).reshape(-1,1))
        D1[Col_Name] = X_scaled
        
        X_scaled = preprocessing.MinMaxScaler().fit_transform(np.array(D2[Col_Name]).reshape(-1,1))
        D2[Col_Name] = X_scaled
    # del Col_Name, X_scaled
    
    D_corr = [D1.corr(), D2.corr()]
    
    P_val = {}
    P_val['1D'] = {}
    P_val['2D'] = {}
    for i in list(D_1D.columns)[3:7]:
        for j in list(D_1D.columns)[-3:]:
            stat, p = stats.ttest_rel(D1[i],D1[j])
            P_val['1D'][i,j] = p
            # print(f"1D : [{i}, {j}] ;- {p}")
            stat, p = stats.ttest_ind(D2[i],D2[j], equal_var=False)
            P_val['2D'][i,j] = p
            # print(f"2D : [{i}, {j}] ;- {p}")
    
    Book = {}
    if SHOW_PLOT:
        for Vy in ['Ob Abundance', 'Settling Time', 'Persistence']:
            f, ax = plt.subplots(2,4, figsize = (30,15), dpi = set_dpi)
            f.tight_layout(pad = 0)
            for D_ind in range(len(D_lst)):
                
                D = D_lst[D_ind]
                
                D_copy = D.copy()
                for Col_Name in list(D.columns):
                    X_scaled = preprocessing.MinMaxScaler().fit_transform(np.array(D_copy[Col_Name]).reshape(-1,1))
                    D_copy[Col_Name] = X_scaled
                del X_scaled
                
                regr_all = LinearRegression().fit(D_copy[['S', 'A', 'C', 'N']],D_copy[Vy])
                r2_all = r2_score(D_copy[Vy], regr_all.predict(D_copy[['S', 'A', 'C', 'N']]))
                
                for V_ind in [0,1,2,3]:
                    V = V_lst[V_ind]
                    X_seq = np.array(D[V]).reshape(-1,1)
                    Y_seq = np.array(D[Vy]).reshape(-1,1)
                    
                    if (Vy == 'Ob Abundance') and (D_ind == 0):
                        Y_seq = (Y_seq - 0) / (14-0)
                    
                    regr = LinearRegression().fit(X_seq,Y_seq)
                    
                    if Vy in ['Abundance','Ob Abundance']:
                        ax[D_ind,V_ind].scatter(X_seq,Y_seq, alpha = 0.05, color = 'black', marker = 1)
                    if Vy in ['Settling Time']:
                        ax[D_ind,V_ind].scatter(X_seq,Y_seq, alpha = 0.10, color = 'black', marker = 2)
                    if Vy in ['Persistence']:
                        ax[D_ind,V_ind].scatter(X_seq,Y_seq, alpha = 0.10, color = 'black', marker = 2)
                    
                    ax[D_ind,V_ind].plot(X_seq,regr.predict(X_seq),color="red", linewidth = 5)
                    
                    r2 = r2_score(Y_seq, regr.predict(X_seq))
                    Book[(D_ind, Vy, V)] = r2
                    
                    ax[D_ind,V_ind].tick_params(labelsize = 40, rotation = 30)
                    ax[D_ind,V_ind].tick_params(axis='y', which='major', pad=5)
                    # plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.3, hspace=0.3)
                    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.15, hspace=0.15)
                    
                    if D_ind == 0:
                        if Vy == 'Abundance':ax[D_ind,V_ind].set_ylim([0,1.0])
                        if Vy == 'Ob Abundance':
                            ax[D_ind,V_ind].set_ylim([0,1.0])
                            ax[D_ind,V_ind].set_yticks(np.arange(0.0,1.05,0.25))
                        if V == "S":
                            if Vy in ['Abundance','Ob Abundance']:
                                ax[D_ind,V_ind].set_ylabel('1-D Model \n'+r"<$X$> $\longrightarrow$", fontsize = 45)
                                # ax[D_ind,V_ind].set_yticks(np.arange(14))
                                # ax[D_ind,V_ind].set_yticklabels(['0','','','','4','','','','8','','','','12',''])
                                f.text(-0.15, 0.5, r"Abundance $\longrightarrow$", va='center', ha='center',rotation = 'vertical', fontsize=66, style='italic')
                            if Vy in ['Settling Time']:
                                ax[D_ind,V_ind].set_ylabel('1-D Model \n'+r"<$ST$> $\longrightarrow$", fontsize = 45)
                                f.text(-0.15, 0.5, r"Settling TIme $\longrightarrow$", va='center', ha='center',rotation = 'vertical', fontsize=66, style='italic')
                            if Vy in ['Persistence']:
                                ax[D_ind,V_ind].set_ylim([0,1.00])
                                ax[D_ind,V_ind].set_ylabel('1-D Model \n'+r"<$P$> $\longrightarrow$", fontsize = 45)
                                f.text(-0.15, 0.5, r"Persistence $\longrightarrow$", va='center', ha='center',rotation = 'vertical', fontsize=66, style='italic')
                            ax[D_ind,V_ind].set_ylabel(r"($R^2$ = " + str(round(r2_all,3)) + ')', fontsize = 40)
                            ax[D_ind,V_ind].yaxis.label.set_color('blue')
                        elif V == "A":
                            ax[D_ind,V_ind].set_xlim([0,20])
                            ax[D_ind,V_ind].set_yticks([])
                        elif V == "C":
                            ax[D_ind,V_ind].set_xlim([0,0.5])
                            ax[D_ind,V_ind].set_yticks([])
                        elif V == "N":
                            ax[D_ind,V_ind].set_yticks([])
                        
                        ax[D_ind,V_ind].set_xticklabels([])
                        f.text(-0.075, 0.75, r"1-D Model", va='center', ha='center',rotation = 'vertical', fontsize=66, style='italic')
                        
                    if D_ind == 1:
                        if Vy == 'Abundance':ax[D_ind,V_ind].set_ylim([0,1.0])
                        if Vy == 'Ob Abundance':
                            ax[D_ind,V_ind].set_ylim([0,1.05])
                            ax[D_ind,V_ind].set_yticks(np.arange(0.0,1.05,0.25))
                        
                        if V == "S":
                            ax[D_ind,V_ind].set_xlabel('\n'+r"Network Size $\longrightarrow$", fontsize = 50)
                            if Vy in ['Abundance','Ob Abundance']:
                                ax[D_ind,V_ind].set_ylabel('2-D Model \n'+r"<$X$> $\longrightarrow$", fontsize = 45)
                            if Vy in ['Settling Time']:
                                ax[D_ind,V_ind].set_ylabel('2-D Model \n'+r"<$ST$> $\longrightarrow$", fontsize = 45)
                            if Vy in ['Persistence']:
                                ax[D_ind,V_ind].set_ylim([0,1.05])
                                ax[D_ind,V_ind].set_ylabel('2-D Model \n'+r"<$P$> $\longrightarrow$", fontsize = 45)
                            ax[D_ind,V_ind].set_ylabel(r"($R^2$ = " + str(round(r2_all,3)) + ')', fontsize = 40)
                            ax[D_ind,V_ind].yaxis.label.set_color('blue')
                            ax[D_ind,V_ind].set_xticks([0,50,100,150,200])
                            
                        elif V == "A":
                            ax[D_ind,V_ind].set_xlabel('\n'+r"Asymmetry $\longrightarrow$", fontsize = 50)
                            ax[D_ind,V_ind].set_yticks([])
                            ax[D_ind,V_ind].set_xlim([0,20])
                        elif V == "C":
                            ax[D_ind,V_ind].set_xlabel('\n'+r"Connectance $\longrightarrow$", fontsize = 50)
                            ax[D_ind,V_ind].set_yticks([])
                            ax[D_ind,V_ind].set_xlim([0,0.5])
                        elif V == "N":
                            ax[D_ind,V_ind].set_xlabel('\n'+r"Nestedness $\longrightarrow$", fontsize = 50)
                            ax[D_ind,V_ind].set_yticks([])
                            ax[D_ind,V_ind].set_xlim([0,0.6])
                        f.text(-0.075, 0.25, r"2-D Model", va='center', ha='center',rotation = 'vertical', fontsize=66, style='italic')
                    ax[D_ind,V_ind].text(0.55,0.05, r'$coef = $' + str(round(regr_all.coef_[V_ind],3)) + '\n' + r'$corr = $' + str(round(D_corr[D_ind].loc[V,Vy],3)), 
                                         transform=ax[D_ind,V_ind].transAxes, 
                                         fontsize = 30, 
                                         color = 'blue',
                                         va='bottom', ha='left', 
                                         bbox = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
                                         )
        
        # del D_lst, V_lst, Vy, f, ax, D_ind, V_ind, X_seq, Y_seq, regr, V, D
    
    return D1, D2, P_val, Book

In [ ]:
# Scatter Plot & Overall Correlation + P value
D1,D2,P, Book = get_ScatterPlot(SHOW_PLOT = True, set_dpi = 100)
D1 = D1.corr()
D2 = D2.corr()

In [6]:
D1

,n,m,L,S,A,C,N,Abundance,Ob Abundance,Settling Time,Persistence
n,1.000000,-0.384974,0.413146,0.388523,-0.694352,-0.188213,-0.315561,-0.198334,-0.197061,0.243649,0.573689
m,-0.384974,1.000000,0.210912,0.700850,0.666530,-0.386878,0.167323,0.303301,0.302522,-0.397929,0.079013
L,0.413146,0.210912,1.000000,0.529882,-0.185619,0.204718,-0.126240,0.682771,0.683005,-0.413270,0.513445
S,0.388523,0.700850,0.529882,1.000000,0.128807,-0.531719,-0.076836,0.149524,0.149730,-0.208977,0.522276
A,-0.694352,0.666530,-0.185619,0.128807,1.000000,0.110045,0.409793,0.250326,0.249106,-0.306747,-0.366074
C,-0.188213,-0.386878,0.204718,-0.531719,0.110045,1.000000,0.265370,0.548397,0.548189,-0.221305,-0.356222
N,-0.315561,0.167323,-0.126240,-0.076836,0.409793,0.265370,1.000000,0.193956,0.193179,-0.341228,-0.247650
Abundance,-0.198334,0.303301,0.682771,0.149524,0.250326,0.548397,0.193956,1.000000,0.999995,-0.564545,0.000500
Ob Abundance,-0.197061,0.302522,0.683005,0.149730,0.249106,0.548189,0.193179,0.999995,1.000000,-0.563858,0.001272
Settling Time,0.243649,-0.397929,-0.413270,-0.208977,-0.306747,-0.221305,-0.341228,-0.564545,-0.563858,1.000000,-0.009238


In [7]:
D2

,n,m,L,S,A,C,N,Abundance,Ob Abundance,Settling Time,Persistence
n,1.000000,-0.288083,0.439336,0.432373,-0.663630,-0.211240,-0.236768,0.113915,0.113917,0.007753,0.280035
m,-0.288083,1.000000,0.333908,0.738908,0.645063,-0.337114,0.184119,-0.607017,-0.607015,0.097463,0.433238
L,0.439336,0.333908,1.000000,0.623546,-0.109051,0.188874,0.056751,0.183307,0.183309,-0.367794,0.413837
S,0.432373,0.738908,0.623546,1.000000,0.140446,-0.466067,0.006770,-0.491421,-0.491418,0.097228,0.604986
A,-0.663630,0.645063,-0.109051,0.140446,1.000000,0.160254,0.376085,-0.507883,-0.507883,-0.092965,0.089448
C,-0.211240,-0.337114,0.188874,-0.466067,0.160254,1.000000,0.356130,0.557897,0.557895,-0.628585,-0.404648
N,-0.236768,0.184119,0.056751,0.006770,0.376085,0.356130,1.000000,-0.032461,-0.032461,-0.332231,-0.029909
Abundance,0.113915,-0.607017,0.183307,-0.491421,-0.507883,0.557897,-0.032461,1.000000,1.000000,-0.291752,-0.394568
Ob Abundance,0.113917,-0.607015,0.183309,-0.491418,-0.507883,0.557895,-0.032461,1.000000,1.000000,-0.291751,-0.394563
Settling Time,0.007753,0.097463,-0.367794,0.097228,-0.092965,-0.628585,-0.332231,-0.291752,-0.291751,1.000000,0.204438
